# SPM batch script generator

### Following scripts are available
#### 1. Import DICOMs
#### 2. Realign & Slice timing
#### 3. Coregister & Segment & Normalize & Smoothing
#### 4. fMRI model specification
#### 5. Contrast Manager

In [ ]:
import tkinter as TK
import os, shutil
import pandas as pd
from glob import glob
from tkinter import filedialog, messagebox
from tqdm.notebook import tqdm
import nibabel as nib

TEMP = TK.Tk(); _ = TEMP.withdraw()

In [ ]:
def create_script(path, fname, info = "%% SPM DICOM Import Batch Maker by Kim Yun Heung\n"):
    File = open(path + '/' + fname,'w')
    File.write(info)

    runner = open(path + '/run' + fname.split('.')[0] + '.m', 'w')
    runner.write("""%% List of open inputs
nrun = 1; %% enter the number of runs here
jobfile = {'%s'};
jobs = repmat(jobfile, 1, nrun);
inputs = cell(0, nrun);
for crun = 1:nrun
end
spm('defaults', 'FMRI');
spm_jobman('run', jobs, inputs{:});
"""%(path + '/' + fname))
    return File

def Ask_directory(Title='Test'):
    Path =  filedialog.askdirectory(title=Title,initialdir=os.getcwd())
    os.chdir(Path)
    return Path

`Main Presets`

In [ ]:
Sequence = { 'Anatomy': 'BRAVO'   ,
             'fMRI'   : 'NEGATIVE' }
Folder_Structure = '/*/*/' # Subject / sequence
SPM_PATH = 'C:/Users/KIM/Documents/MATLAB/spm12/'
base_dir = os.getcwd()

script_path = Ask_directory('Where to save script files?')

In [ ]:
print(script_path)

## Script #1 Import DICOMs

`Presets`

In [ ]:
def script1_writer(index, path, destination , File):
    File.write('matlabbatch{%d}.spm.util.import.dicom.data = {\n'%(index))
    for dcm in sorted(glob(path + '/*')): 
        File.write("	'" + dcm + "'\n")
    File.write('};')
    File.write("""
matlabbatch{%d}.spm.util.import.dicom.root = 'flat';
matlabbatch{%d}.spm.util.import.dicom.outdir = {'%s'};
matlabbatch{%d}.spm.util.import.dicom.protfilter = '.*';
matlabbatch{%d}.spm.util.import.dicom.convopts.format = 'nii';
matlabbatch{%d}.spm.util.import.dicom.convopts.meta = 0;
matlabbatch{%d}.spm.util.import.dicom.convopts.icedims = 0;\n\n"""%(index, index, destination, index, index, index, index))


In [ ]:
dicom_path = Ask_directory('Where are the DICOM files?')
nifti_path = Ask_directory('Where to save NiFTI files?')
try: os.mkdir(nifti_path + '/NIFTI')
except: print('Warning: The folder is already exist! Your data will be overwritten!')

In [ ]:
print(dicom_path)
print(nifti_path)

In [ ]:
Anatomy_list, Anatomy_nifti = pd.DataFrame(), pd.DataFrame()
Anatomy_list['Path'], Anatomy_nifti['Path'] = None, None
fMRI_list, fMRI_nifti = pd.DataFrame(), pd.DataFrame()
fMRI_list['Path'], fMRI_nifti['Path'] = None, None
index = 0

for subject in glob(dicom_path + '/*'):
    input_conditions = [False, False]
    if os.path.isdir(subject):
        Subject = subject.split('\\')[-1]
        for sequence in glob(subject+'/*'):
            if   Sequence['Anatomy'] in sequence.upper(): input_conditions[0] = sequence#; print(subject)
            elif Sequence['fMRI'] in sequence.upper()   : input_conditions[1] = sequence#; print(subject)
            else: pass
        if False not in input_conditions:
            try: os.mkdir(nifti_path + '/NIFTI/' + Subject)
            except: pass
            for seq in [Sequence['Anatomy'], Sequence['fMRI']]:
                try: os.mkdir(nifti_path + '/NIFTI/' + Subject + '/' + seq)
                except: pass
            Anatomy_list.loc[index] = input_conditions[0]
            Anatomy_nifti.loc[index] = nifti_path + '/NIFTI/' + Subject + '/' + Sequence['Anatomy']
            fMRI_list.loc[index] = input_conditions[1]
            fMRI_nifti.loc[index] = nifti_path + '/NIFTI/' + Subject + '/' + Sequence['fMRI'] 
            index += 1

In [ ]:
print('Anatomy total:',len(Anatomy_nifti))
print('fMRI total:'   ,len(fMRI_nifti)  )

In [ ]:
script1 = create_script(script_path, 'script1.m')
print('Writing script...', end=' ')

idx = 0
for i in tqdm(range(0, len(Anatomy_list['Path'])*2,2)):
    script1_writer(i+1, Anatomy_list['Path'].loc[idx], Anatomy_nifti['Path'][idx], File=script1)
    script1_writer(i+2, fMRI_list['Path'].loc[idx], fMRI_nifti['Path'][idx], File=script1)
    idx += 1
    
print('Done!'); script1.close()

## Script #2 Realign & Slice timing

`Script #2 presets`

In [ ]:
realign = {
    ##Estimation Options##
    'Quality'      : 0.9,       #0~1, higher the better, lower the faster
    'Separation'   : 4,         #The separation (in mm) between the points sampled in the reference image
    'Smoothing'    : 5,         #The FWHM of the Gaussian smoothing kernel (mm) applied to the images before estimating
    'Num_Passes'   : 1,         #0: Register to first, 1: Register to mean
    'Interpolation': 2,         #0: Trilinear, 2~7: n-th Degree B-Spline
    'Wrap'         : '[0 0 0]', #[X, Y, Z], 0: no wrapping, 1: wrapping for the P.E direction
    ##Reslice Options##
    'Masking'      : 1,       #Searching through the whole time series looking for voxels which need to be 
                              #sampled from outside the original images.
    'fname_prefix' : 'r'      #default: 'r'
}

sliceTiming = {
    'Number of slices': 'AUTO',  #Number of slices will be filled automatically.
    'TR'              : 3     ,  #n * 1000ms
    'TA'              : 'AUTO',  #TA will be filled automatically.
    'Slice_order'     : 'AUTO',  #slice order will be filled automatically
    'Reference slice' : 1     ,  #If slice times are provided instead of slice indices in the previous item, this value should
                                 #represent a reference time (in ms) instead of the slice index of the reference slice.
    'fname_prefix' : 'a'         #default: 'a'
}

In [ ]:
def script2_writer(index, path, realign, sliceT, File):
    LIST = sorted(glob(path + '/s*.nii'))
    print(path + '/*')
    reference = nib.load(LIST[0]).get_fdata()
    nslices   = reference.shape[-1]
    File.write("matlabbatch{%d}.spm.spatial.realign.estwrite.data = {\n"%(index))
    File.write('                                                    {\n')
    for nif in LIST: File.write("                                                    '" + nif + "'\n")
    TA = sliceT['TR'] - sliceT['TR']/nslices
    slice_order = str([i for i in range(1, nslices if nslices%2==0 else nslices+1 ,2)] + [i for i in range(2, nslices+1 if nslices%2==0 else nslices, 2)]).replace(',','')
    File.write('                                                    }\n')
    File.write("                                                    }';\n")
    File.write("matlabbatch{%d}.spm.spatial.realign.estwrite.eoptions.quality = %s;\n"%(  index, realign['Quality']))
    File.write("matlabbatch{%d}.spm.spatial.realign.estwrite.eoptions.sep = %s;\n"%(      index, realign['Separation']))
    File.write("matlabbatch{%d}.spm.spatial.realign.estwrite.eoptions.fwhm = %s;\n"%(     index, realign['Smoothing']))
    File.write("matlabbatch{%d}.spm.spatial.realign.estwrite.eoptions.rtm = %s;\n"%(      index, realign['Num_Passes']))
    File.write("matlabbatch{%d}.spm.spatial.realign.estwrite.eoptions.interp = %s;\n"%(   index, realign['Interpolation']))
    File.write("matlabbatch{%d}.spm.spatial.realign.estwrite.eoptions.wrap = %s;\n"%(     index, realign['Wrap']))
    File.write("matlabbatch{%d}.spm.spatial.realign.estwrite.eoptions.weight = '';\n"%(   index  ))

    File.write("matlabbatch{%d}.spm.spatial.realign.estwrite.roptions.which = [2 1];\n"%( index  )) #[All images, Mean image]
    File.write("matlabbatch{%d}.spm.spatial.realign.estwrite.roptions.interp = 4;\n"%(    index  ))
    File.write("matlabbatch{%d}.spm.spatial.realign.estwrite.roptions.wrap = %s;\n"%(     index, realign['Wrap']))
    File.write("matlabbatch{%d}.spm.spatial.realign.estwrite.roptions.mask = %s;\n"%(     index, realign['Masking']))
    #File.write("matlabbatch{%d}.spm.spatial.realign.estwrite.roptions.prefix = '%s';\n"%( index, realign['fname_prefix']))

    File.write(
"""matlabbatch{%d}.spm.temporal.st.scans{1}(1) = cfg_dep('Realign: Estimate & Reslice: Resliced Images (Sess 1)', substruct('.','val', '{}',{%d}, '.','val', '{}',{1}, '.','val', '{}',{1}, '.','val', '{}',{1}), substruct('.','sess', '()',{1}, '.','rfiles'));\n"""%(index+1,index))
    File.write("matlabbatch{%d}.spm.temporal.st.nslices = %s;\n"%( index+1, nslices))
    File.write("matlabbatch{%d}.spm.temporal.st.tr = %d;\n"%(      index+1, sliceT['TR']))
    File.write("matlabbatch{%d}.spm.temporal.st.ta = %s;\n"%(      index+1, TA))
    File.write("matlabbatch{%d}.spm.temporal.st.so = %s\n"%(       index+1, slice_order))
    File.write("matlabbatch{%d}.spm.temporal.st.refslice = %s;\n"%(index+1, sliceTiming['Reference slice']))
    File.write("matlabbatch{%d}.spm.temporal.st.prefix = '%s';\n"%(index+1, sliceTiming['fname_prefix']))
    File.write("\n")

In [ ]:
script2 = create_script(script_path, 'script2.m', info="%% SPM Realign & Slice Timing Batch Maker by Kim Yun Heung\n")
print('Writing script...')
idx = 0
for i in tqdm(range(0, len(fMRI_list['Path'])*2,2)):
    #print(fMRI_nifti['Path'][idx])
    script2_writer(i+1, fMRI_nifti['Path'][idx], realign, sliceTiming, File=script2)
    idx += 1
#break
script2.close()
print('Done!')

## Script #3 Coregister & Segment & Normalize & Smoothing

In [ ]:
auto_backup = messagebox.askyesno('Auto backup',"When you are doing coregister sequence, all of the T1 images will be overwritten with coregistered T1 images. Click 'yes' to backup all of the T1 images before continue.")

if auto_backup:
    directory = filedialog.askdirectory(title='Where to save?',initialdir=os.getcwd())
    try: os.mkdir(directory+'/T1_backup')
    except: print('Warning: The folder is already exist! Your data will be overwritten!')
    
    for Folder in Anatomy_nifti['Path']:
        Subject = Folder.split('/')[-2]
        try: os.mkdir(directory + '/T1_backup/' + Subject)
        except: pass
        shutil.copy(glob(Folder+'/*.nii')[0], directory + '/T1_backup/' + Subject)

`Script #3 Presets`

In [ ]:
coregister = {
    'Objective_Function' : 'nmi',   #nmi: Normalized Mutual Information
    'Separation'         : '[4 2]', #The average distance between sampled points (in mm)
    'Tolerance'          : '[0.02 0.02 0.02 0.001 0.001 0.001 0.01 0.01 0.01 0.001 0.001 0.001]',      
                                    #Iterations stop when differences between successive estimates are less thean the required tolerance
    'Smoothing'          : '[7 7]'  #Gaussian smoothing to apply to the 256x256 joint histogram.
}

Segment = {
    'Bias_regularization'     : 0.001,   #0 / 0.00001 / 0.0001 / 0.01 / 0.1 / 1 / 10 ==> ex) If bias is very little, set lower value
    'Bias_FWHM'               : 60,      #mm, 30 / 40 / 50 / 60 / 70 / 80 / 90 / 100 / 110 / 120 / 130 / 140 / 150 / None
    'Save_corrected_and_field': '[0 1]', #[Field, corrected]
    'Num_gaussians'           : [1,1,2,3,4,2], #0~8, 
                                         #Used to represent the intensity distribution for each tissue class can be greater than one
    'Native_Tissue'           : [ '[1 1]', '[1 1]', '[1 1]', '[1 0]', '[1 0]', '[0 0]' ], #[Native, Dartel]
                                         #Produce a tissue class image (C*) that is in alignment with the original.[Dartel toolbox(rc*)]
    'Warped_Tissue'           : [ '[1 0]', '[1 0]', '[1 0]', '[1 0]', '[1 0]', '[1 0]' ], #[Modulated, Unmodulated]
                                         #Produce spatially normalised versions of the tissue class. (mwc* and wc*)
    'MRF_Filter'              : 1,       #Strength of MRF
                                         #Cleanup procedure for tissue class images with few iterations of a simple Markov Random Field 
    'Clean_up'                : 1,       #0: Don't cleanup, 1: Light Clean, 2: Thorough clean
    'Warping_regularization'  : '[0 0.001 0.5 0.05 0.2]', #Read SPM manual for detailed instruction
                                         #Measure of similarity between images and measure of the roughness of the deformation
    'Affine_Regularization'   : 'eastern', 
    #No Affine Registration: ''
    #No regularisation: 'none'
    #ICBM space template - East Asian Brains: 'eastern'
    #ICBM space template - European: 'mni'
    #Average sized template: 'subj'
    'Smoothness'              : 0,         #Derive a fudge factor to account for correlations between neighbouring voxels 
    'Sampling_distance'       : 3,         #Encodes the approximate distance between smapled points when estimating the model parameters
    'Deformation_Fields'      : '[0 1]' #[Inverse, Forward]
                                    #Forward: spatially normalizing image to MNI / Inverse: spatailly normalizing GIFTI surface files
}

Normalize = {
    'Bounding_box' : [-78, -112, -70, 78, 76, 85],
    'Voxel_sizes'  : '[2 2 2]',
    'Interpolation': 4,
    'fname_prefix' : 'w',
    }

Smoothing = {
    'FWHM'         : '[10 10 10]',
    'Data_type'    : 0        , #0: Same dtype as original, 1: UINT8, 2: INT16, 3: INT32, 4: Float32, 5: Float64
    'Implicit_mask': 0        , 
    'fname_prefix' : 'FINAL_' 
}

In [ ]:
def script3_writer(index, path1, path2, coregister, segment, normalize, smoothing, FILE):
    global SPM_PATH
    Mean_EPI = glob(path2 + '/mean*')[0]
    anatomy  = glob(path1 + '/*')[0]
    FILE.write("matlabbatch{%d}.spm.spatial.coreg.estimate.ref    = {'%s,1'};\n"%(index, Mean_EPI))
    FILE.write("matlabbatch{%d}.spm.spatial.coreg.estimate.source = {'%s,1'};\n"%(index, anatomy))
    FILE.write("matlabbatch{%d}.spm.spatial.coreg.estimate.other  = {''};\n"%(index))
    ########################################################################################################################################
    FILE.write("matlabbatch{%d}.spm.spatial.coreg.estimate.eoptions.cost_fun = '%s';\n"%(index, coregister['Objective_Function']))
    FILE.write("matlabbatch{%d}.spm.spatial.coreg.estimate.eoptions.sep = %s;\n"%(index, coregister['Separation']))
    FILE.write("matlabbatch{%d}.spm.spatial.coreg.estimate.eoptions.tol = %s;\n"%(index, coregister['Tolerance']))
    FILE.write("matlabbatch{%d}.spm.spatial.coreg.estimate.eoptions.fwhm = %s;\n"%(index, coregister['Smoothing']))
########################################################################################################################################
    FILE.write("matlabbatch{%d}.spm.spatial.preproc.channel.vols(1) = cfg_dep('Coregister: Estimate: Coregistered Images', substruct('.','val', '{}',{%d}, '.','val', '{}',{1}, '.','val', '{}',{1}, '.','val', '{}',{1}), substruct('.','cfiles'));\n"%(index+1,index))
    FILE.write("matlabbatch{%d}.spm.spatial.preproc.channel.biasreg = %s;\n"%(index+1, segment['Bias_regularization']))
    FILE.write("matlabbatch{%d}.spm.spatial.preproc.channel.biasfwhm = %s;\n"%(index+1,segment['Bias_FWHM']))
    FILE.write("matlabbatch{%d}.spm.spatial.preproc.channel.write = %s;\n"%(index+1,segment['Save_corrected_and_field']))
    for idx in range(1,7):
        FILE.write("matlabbatch{%d}.spm.spatial.preproc.tissue(%d).tpm = {'%stpm/TPM.nii,%d'};\n"%(index+1, idx, SPM_PATH, idx))
        FILE.write("matlabbatch{%d}.spm.spatial.preproc.tissue(%d).ngaus = %d;\n"%(index+1, idx, segment['Num_gaussians'][idx-1]))
        FILE.write("matlabbatch{%d}.spm.spatial.preproc.tissue(%d).native = %s;\n"%(index+1, idx, segment['Native_Tissue'][idx-1]))
        FILE.write("matlabbatch{%d}.spm.spatial.preproc.tissue(%d).warped = %s;\n"%(index+1, idx, segment['Warped_Tissue'][idx-1]))
    FILE.write("matlabbatch{%d}.spm.spatial.preproc.warp.mrf = %s;\n"%(index+1, segment['MRF_Filter']))
    FILE.write("matlabbatch{%d}.spm.spatial.preproc.warp.cleanup =  %s;\n"%(index+1, segment['Clean_up']))
    FILE.write("matlabbatch{%d}.spm.spatial.preproc.warp.reg = %s;\n"%(index+1, segment['Warping_regularization']))
    FILE.write("matlabbatch{%d}.spm.spatial.preproc.warp.affreg = '%s';\n"%(index+1, segment['Affine_Regularization']))
    FILE.write("matlabbatch{%d}.spm.spatial.preproc.warp.fwhm = %s;\n"%(index+1, segment['Smoothness']))
    FILE.write("matlabbatch{%d}.spm.spatial.preproc.warp.samp = %s;\n"%(index+1, segment['Sampling_distance']))
    FILE.write("matlabbatch{%d}.spm.spatial.preproc.warp.write = %s;\n"%(index+1, segment['Deformation_Fields']))
    FILE.write("matlabbatch{%d}.spm.spatial.preproc.warp.vox = NaN;\n"%(index+1))
    FILE.write("matlabbatch{%d}.spm.spatial.preproc.warp.bb = [NaN NaN NaN\n"%(index+1))
    FILE.write("                                              NaN NaN NaN];\n")
########################################################################################################################################
    FILE.write("matlabbatch{%d}.spm.spatial.normalise.write.subj.def(1) = cfg_dep('Segment: Forward Deformations', substruct('.','val', '{}',{%d}, '.','val', '{}',{1}, '.','val', '{}',{1}), substruct('.','fordef', '()',{':'}));\n"%(index+2,index+1))
    FILE.write("matlabbatch{%d}.spm.spatial.normalise.write.subj.resample = {\n"%(index+2))
    for imgs in glob(path2 + '/ars*'): FILE.write("                                                            '" + imgs + ",1'\n")
    FILE.write("                                                            };\n")
    FILE.write("matlabbatch{%d}.spm.spatial.normalise.write.woptions.bb = [ %d %d %d\n"%(index+2,   normalize['Bounding_box'][0],
                                                                                                    normalize['Bounding_box'][1],
                                                                                                    normalize['Bounding_box'][2]))
    FILE.write("                                                            %d %d %d];\n"%(         normalize['Bounding_box'][-3],
                                                                                                    normalize['Bounding_box'][-2],
                                                                                                    normalize['Bounding_box'][-1]))
    FILE.write("matlabbatch{%d}.spm.spatial.normalise.write.woptions.vox = %s;\n"%(index+2, normalize['Voxel_sizes']))
    FILE.write("matlabbatch{%d}.spm.spatial.normalise.write.woptions.interp = %d;\n"%(index+2, normalize['Interpolation']))
    FILE.write("matlabbatch{%d}.spm.spatial.normalise.write.woptions.prefix = '%s';\n"%(index+2, normalize['fname_prefix']))
########################################################################################################################################
    FILE.write("matlabbatch{%d}.spm.spatial.smooth.data(1) = cfg_dep('Normalise: Write: Normalised Images (Subj 1)', substruct('.','val', '{}',{%d}, '.','val', '{}',{1}, '.','val', '{}',{1}, '.','val', '{}',{1}), substruct('()',{1}, '.','files'));\n"%(index+3,index+2))
    FILE.write("matlabbatch{%d}.spm.spatial.smooth.fwhm = %s;\n"%(index+3, smoothing['FWHM']))
    FILE.write("matlabbatch{%d}.spm.spatial.smooth.dtype = %d;\n"%(index+3, smoothing['Data_type']))
    FILE.write("matlabbatch{%d}.spm.spatial.smooth.im = %d;\n"%(index+3, smoothing['Implicit_mask']))
    FILE.write("matlabbatch{%d}.spm.spatial.smooth.prefix = '%s';\n\n"%(index+3, smoothing['fname_prefix']))

In [ ]:
script3 = create_script(script_path, 'script3.m')
print('Writing script...')
idx = 0
for i in tqdm(range(0, len(fMRI_list['Path'])*4, 4 ) ):
    script3_writer(i+1, Anatomy_nifti['Path'][idx], fMRI_nifti['Path'][idx], coregister, Segment, Normalize, Smoothing, FILE=script3)
    idx += 1

script3.close()
print('Done!')

In [ ]:
fMRI_nifti['Path'].loc[0]

## Script #4 fMRI model specification

`Script #4 presets`

In [ ]:
Use_art = False #If the value is set true, art(ARtifact detection Tool) batch generator will be run.

In [ ]:
Fixed_path = False
SPM_MAT_list = []
Specification = {
    'SPM.mat_dir': 'D:/Workspace/TEST/Control/SPM_MAT' if Fixed_path else filedialog.askdirectory(title='Where to save SPM.mat files?', 
                                                                                            initialdir=os.getcwd()),
    'Units_for_design'    : 'secs', #'secs'
    'Interscan_interval'  : 3,  #TR
    'Microtime_resolution': 16,
    'Microtime_onset'     : 8,
    'Session_total'       : 3,
    'Session_info'        :[ 
            #Session name             (ex) If there are two sessions, [ 'Task', 'Fixation' ]
            ['A', 'B', 'Fixation'], 
            #Oneset times per session (ex) If there are two sessions, [ [0, 84, 120, 156], [36, 72, 132, 144, 216] ]  
            [  [0.75, 5.5,10.25,15,19.75,24.5,29.25,34,38.75,43.5,48.25,53,228.75,233.5,238.25,243,247.75,252.5,257.25,262,266.75,
                271.5,276.25,281,342.75,347.5,352.25,357,361.75,366.5,371.25,376,380.75,385.5,390.25,395],
               [57.75,62.5,67.25,72,76.75,81.5,86.25,91,95.75,100.5,105.25,110,171.75,176.5,181.25,186,190.75,195.5
               ,200.25,205,209.75,214.5,219.25,224,399.75,404.5,409.25,414,418.75,423.5,428.25,433,437.75,442.5,447.25,452],
               [114,285,456]            ],
            #Duration                 (ex) If there are two sessions, [ 10, 5 ] 
               [3.5, 3.5, 57.0], 

            #[], #Time modulations, 0: None, 1~6: order of time modulation (ex) If there are two sessions, [0, 0] ==> Working on it :<
            #[], #Parametric modulations   ==> Working on it :<
            #[], #Orthogonalise modulations, 0: no, 1: yes, (ex) If there are two sessions, [1, 1]] ==> Working on it :<
    ],
    #'Multiple_conditions': '', Working on it :<
    #'Regressors'
    'Multiple_regressor'  : 'rp', #Type prefix of multiple regressor file
    'High_Pass_filter'    : 128,
    #'Canionical_HRF'      : '[0 0]', Working on it :<
    #'Global_normalization': 'None', Working on it :<
    'Masking_threshold'   : 0.8,
    #'Explicit_mask'      : '', Working on it :<
    #'Serial_correlations : 'AR(1)', Working on it :<
}

In [ ]:
print(Specification['SPM.mat_dir'])

In [ ]:
def script4_writer(index, fMRI_path, specification, FILE, final_prefix='FINAL_'):
    try: os.mkdir(specification['SPM.mat_dir']+'/'+ fMRI_path.split('/')[-2])
    except: pass
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.dir = {'%s'};\n"%(index, specification['SPM.mat_dir']+'/'+ fMRI_path.split('/')[-2]))
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.timing.units = '%s';\n"%(index, specification['Units_for_design']))
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.timing.RT = %d;\n"%(index, specification['Interscan_interval']))
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.timing.fmri_t = %d;\n"%(index, specification['Microtime_resolution']))
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.timing.fmri_t0 = %d;\n"%(index,specification['Microtime_onset']))
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.sess.scans = {\n"%(index))
    for imgs in glob(fMRI_path + '/' + final_prefix + '*.nii'):
        FILE.write("                                                    '%s,1'\n"%(imgs))
    FILE.write("                                                    };\n")
    
    for sess in range(specification['Session_total']):
        FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.sess.cond(%d).name = '%s';\n"%(index, sess+1, specification['Session_info'][0][sess]))
        FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.sess.cond(%d).onset = [%d\n"%(index, sess+1,specification['Session_info'][1][sess][0]))
        for i in range( 1, len( specification['Session_info'][1][sess] ) ):
            FILE.write("                                                         %d\n"%(specification['Session_info'][1][sess][i]))
        FILE.write("                                                         %d];\n"%(specification['Session_info'][1][sess][-1]))
        FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.sess.cond(%d).duration = %d;\n"%(index, sess+1, specification['Session_info'][2][sess]))
        FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.sess.cond(%d).tmod = 0;\n"%(index, sess+1))
        FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.sess.cond(%d).pmod = struct('name', {}, 'param', {}, 'poly', {});\n"%(index, sess+1))
        FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.sess.cond(%d).orth = 1;\n\n"%(index, sess+1))

    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.sess.multi = {''};\n"%(index))
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.sess.regress = struct('name', {}, 'val', {});\n"%(index))
    print(glob(fMRI_path+'/'+specification['Multiple_regressor']+'*')[0])
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.sess.multi_reg = {'%s'};\n"%(index,glob(fMRI_path+'/'+specification['Multiple_regressor']+'*')[0]))
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.sess.hpf = %s;\n"%(index, specification['High_Pass_filter']))
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.fact = struct('name', {}, 'levels', {});\n"%(index))
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.bases.hrf.derivs = [0 0];\n"%(index))
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.volt = 1;\n"%(index))
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.global = 'None';\n"%(index))
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.mthresh = %s;\n"%(index,  specification['Masking_threshold']))
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.mask = {''};\n"%(index))
    FILE.write("matlabbatch{%d}.spm.stats.fmri_spec.cvi = 'AR(1)';\n"%(index))

    return specification['SPM.mat_dir']+'/'+ fMRI_path.split('/')[-2] + '/SPM.mat'

In [ ]:
script4 = create_script(script_path, 'script4.m')
print('Writing script...')
idx = 0
for i in tqdm(range(0, len(fMRI_list['Path']) ) ):
    rtn = script4_writer(i+1, fMRI_nifti['Path'][idx], Specification, FILE=script4)
    SPM_MAT_list.append( rtn )
    idx += 1

script4.close()
print('Done!')

## Script #5 Model estimation

In [ ]:
def script5_writer(index, SPM_MAT_path, FILE):
   FILE.write("matlabbatch{%d}.spm.stats.fmri_est.spmmat = {'%s'};\n"%(index, SPM_MAT_path))
   FILE.write("matlabbatch{%d}.spm.stats.fmri_est.write_residuals = 0;\n"%(index) )
   FILE.write("matlabbatch{%d}.spm.stats.fmri_est.method.Classical = 1;\n"%(index) )

In [ ]:
script5 = create_script(script_path, 'script5.m')
print('Writing script...')
idx = 0
for i in tqdm(range(0, len(SPM_MAT_list) ) ):
    #if 'KJH' not in SPM_MAT_list[i]:
    script5_writer(idx+1, SPM_MAT_list[i], FILE=script5)
    idx += 1

script5.close()
print('Done!')

## Script #6 Contrast Manager

In [ ]:
print(Specification['Session_info'][0])

In [ ]:
Contrast = {
    'A-B'   : '[-1 1 0]',
    'A-Fix' : '[0 1 -1]',
    'B-Fix' : '[1 0 -1]',
}

In [ ]:
try: os.mkdir(script_path+'/contrast_script')
except: pass

In [ ]:
def create_script_for6(path, idx):
    runner = open(path + '/run_job_' + '{}.m'.format(idx), 'w')
    runner.write("""%% List of open inputs
nrun = 1; %% enter the number of runs here
jobfile = {'%s'};
jobs = repmat(jobfile, 1, nrun);
inputs = cell(0, nrun);
for crun = 1:nrun
end
spm('defaults', 'FMRI');
spm_jobman('run', jobs, inputs{:});
"""%(path + '/job_{}.m'.format(idx) ))

def script6_writer(index, SPM_MAT_path, contrast, path):
    with open(path+'/job_{}.m'.format(index),'w') as FILE:
        FILE.write("matlabbatch{1}.spm.stats.con.spmmat = {'%s'};\n"%(SPM_MAT_path))
        for Name, seq in zip(contrast, range(1,len(contrast)+1)):
            FILE.write("matlabbatch{1}.spm.stats.con.consess{%d}.tcon.name = '%s';\n"%(seq,Name) )
            FILE.write("matlabbatch{1}.spm.stats.con.consess{%d}.tcon.weights = %s;\n"%(seq,contrast[Name]) )
            FILE.write("matlabbatch{1}.spm.stats.con.consess{%d}.tcon.sessrep = 'none';\n"%(seq) )
        FILE.write("matlabbatch{1}.spm.stats.con.delete = 1;\n")

In [ ]:
idx = 0
for i in tqdm(range(0, len(SPM_MAT_list))):
    #if 'KJH' not in SPM_MAT_list[i]:
    create_script_for6(script_path+'/contrast_script', idx)
    script6_writer(idx, SPM_MAT_list[i], Contrast, script_path+'/contrast_script')
    idx += 1
print('Done!')